In [1]:
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session
from dotenv import load_dotenv
import os

load_dotenv()


connection_parameters = {
   "account": os.getenv('account_snow'),
   "user": os.getenv('user_snow'),
   "password":os.getenv('password_snow'),
   "role": "ACCOUNTADMIN",
   "database": "NEW_DB", 
   "schema": "PUBLIC"
  
}  

session = Session.builder.configs(connection_parameters).create()

In [2]:
from snowflake.snowpark.functions import when_matched, when_not_matched
from snowflake.snowpark.types import IntegerType, StringType, StructField, StructType
schema = StructType([StructField("key", IntegerType()), StructField("value", StringType())])
target_df = session.create_dataframe([(10, "old"), (10, "too_old"), (11, "old")], schema=schema)
target_df.write.save_as_table("my_table", mode="overwrite", table_type="temporary")
target = session.table("my_table")

In [3]:
source = session.create_dataframe([(10, "new"), (12, "new"), (13, "old")], schema=schema)
target.merge(source, (target["key"] == source["key"]) & (target["value"] == "too_old"),
             [when_matched().update({"value": source["value"]}), when_not_matched().insert({"key": source["key"]})])

MergeResult(rows_inserted=2, rows_updated=1, rows_deleted=0)

In [4]:
target.sort("key", "value").collect()

[Row(KEY=10, VALUE='new'),
 Row(KEY=10, VALUE='old'),
 Row(KEY=11, VALUE='old'),
 Row(KEY=12, VALUE=None),
 Row(KEY=13, VALUE=None)]

In [6]:
target.merge(source, (target["key"] == source["key"]) & (target["value"] == "too_old"),
             [when_matched().update({"value": source["value"]}), when_not_matched().insert({"key": source["key"]})])

MergeResult(rows_inserted=3, rows_updated=0, rows_deleted=0)